In [ ]:
!pip install huggingface-hub transformers sentencepiece protobuf accelerate

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git

In [ ]:
%cd llama.cpp

In [ ]:
!make clean

In [ ]:
!make clean

In [ ]:
!pip install torch transformers sentencepiece

In [ ]:
!pip install huggingface-hub


In [ ]:
!huggingface-cli login

In [ ]:
!huggingface-cli download google/functiongemma-270m-it \
  --local-dir functiongemma-270m-it \
  --local-dir-use-symlinks False

In [ ]:
!python convert_hf_to_gguf.py \
  functiongemma-270m-it \
  --outfile functiongemma-270m-it.f16.gguf \
  --outtype f16

In [ ]:
!./quantize \
  functiongemma-270m-it.f16.gguf \
  functiongemma-270m-it.Q4_K_M.gguf \
  Q4_K_M

In [ ]:
!mkdir -p build

In [ ]:
%cd build

In [ ]:
!cmake .. \
  -DCMAKE_BUILD_TYPE=Release \
  -DLLAMA_BUILD_TOOLS=ON \
  -DLLAMA_BUILD_EXAMPLES=ON \
  -DLLAMA_CUDA=OFF

In [ ]:
!cmake --build . --config Release -j$(nproc)

In [ ]:
!./bin/llama-quantize \
  ../functiongemma-270m-it.f16.gguf \
  ../functiongemma-270m-it.Q4_K_M.gguf \
  Q4_K_M

In [ ]:
!ls bin

In [ ]:
!./bin/llama-cli \
  -m ../functiongemma-270m-it.Q4_K_M.gguf \
  -p "who are you" \
  -t 4 \
  -c 2048

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
repo_id = "ajay2708/functiongemma-270m-it.Q4_K_M.gguf"
api.create_repo(repo_id, exist_ok=True, repo_type="model")


In [ ]:
api.upload_file(
    path_or_fileobj="../functiongemma-270m-it.Q4_K_M.gguf",
    path_in_repo="functiongemma-270m-it.Q4_K_M.gguf", # The file name it will have in the repo
    repo_id=repo_id,
)
